In [1]:
import csv
import os
import numpy as np
from envs.utils import printoptions


def get_stats(algorithm="mceirl", env="room", path="./results/use_prior"):
    results_list=[]
    for file in os.listdir(path):
        if "use_prior" in file:
            if algorithm in file and env in file:
                #print(os.path.join("./results", file))
                
                with open(os.path.join(path, file), 'rt') as f:
                    reader = csv.reader(f)
                    list_results = list(reader)[1::]
                    list_rewards = []
                    for res in list_results:
                        list_rewards.append(float(res[1]))
                    list_rewards = np.asarray(list_rewards)
                    
                    k_val = file.split('-k=', 1)[-1]
                    k_val = k_val.split('-v')[0]
                    
                    results_list.append([float(k_val), np.mean(list_rewards), np.std(list_rewards)])
    results_list = np.asarray(results_list)
    # return a list sorted by the value of k
    return results_list[results_list[:,0].argsort()]
                    

In [2]:
with printoptions(precision=4, suppress=True):
    print("   k \t mean \t std")
    print(get_stats("mceirl", "room"))

   k 	 mean 	 std
[[ 0.0625  0.      0.    ]
 [ 0.125   0.      0.    ]
 [ 0.25   -0.1111  0.    ]
 [ 0.5     1.      0.    ]
 [ 1.      0.      0.    ]
 [ 2.      0.      0.    ]
 [ 4.      0.      0.    ]
 [ 8.      0.      0.    ]]


In [3]:
with printoptions(precision=4, suppress=True):
    print("  k \t mean \t std")
    print(get_stats("sampling", "room"))

  k 	 mean 	 std
[[0.0625 0.7778 0.4444]
 [0.125  1.     0.    ]
 [0.25   1.     0.    ]
 [0.5    0.6    0.4899]
 [1.     0.4    0.4899]
 [2.     0.     0.    ]
 [4.     0.     0.    ]
 [8.     0.2    0.4   ]]


In [4]:
with printoptions(precision=4, suppress=True):
    print("  k \t mean \t std")
    print(get_stats("mceirl", "apples"))

  k 	 mean 	 std
[[0.0625 0.6031 0.    ]
 [0.0625 0.548  0.0067]
 [0.125  0.6031 0.    ]
 [0.25   0.6031 0.    ]
 [1.     0.8998 0.1077]]


In [5]:
with printoptions(precision=4, suppress=True):
    print("  k \t mean \t std")
    print(get_stats("sampling", "apples"))

  k 	 mean 	 std
[[0.0625 0.802  0.1661]
 [0.125  0.802  0.1661]]
